#**Chat with Multiple Documents using LangChain, Llama3.1 & Chroma**

**Step # 01: Install All the Required Packages**

In [1]:
!pip install -qU langchain-community
!pip install -qU langchain-groq
!pip install -qU langchain
!pip install -qU langchain-chroma
!pip install -qU langchain-huggingface
!pip install -qU pypdf
!pip install -qU PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Gett

**Step 02: Import All the Required Libraries**

In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain_chroma import Chroma
import os

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
os.environ["GROQ_API_KEY"] = "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"

In [5]:
!mkdir Documents

In [6]:
!gdown "https://drive.google.com/uc?id=1ce0NdDwTnxq_JCWKkgfhrs5JpTZEoQUR&confirm=t" -O "/content/Documents/yolov7.pdf"
!gdown "https://drive.google.com/uc?id=1Xes_50KBdNgspsmXUjaB9lI56AJE0vaR&confirm=t" -O "/content/Documents/yolov10.pdf"

Downloading...
From: https://drive.google.com/uc?id=1ce0NdDwTnxq_JCWKkgfhrs5JpTZEoQUR&confirm=t
To: /content/Documents/yolov7.pdf
100% 2.27M/2.27M [00:00<00:00, 202MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Xes_50KBdNgspsmXUjaB9lI56AJE0vaR&confirm=t
To: /content/Documents/yolov10.pdf
100% 1.17M/1.17M [00:00<00:00, 153MB/s]


**Step 03: Extract Text from the PDF Files**

**Indexing: Load**

In [7]:
def extractText(directorypath):
  documents = []
  for filename in os.listdir(directorypath):
    if filename.endswith(".pdf"):
      pdf_path = os.path.join(directorypath, filename)
      loader = PyPDFLoader(pdf_path)
      documents.extend(loader.load())
  return documents


**Step 04: Split the Text into Smaller Text Chunks**


**Indexing: Split**

In [8]:
def textSplitter(text):
  text_splitter = RecursiveCharacterTextSplitter(
     chunk_size = 1000, chunk_overlap = 200, add_start_index = True
  )
  textSplit = text_splitter.split_documents(text)
  print("Length of Text Chunks", len(textSplit))
  print("Text Chunk No. 10", textSplit[10])
  print("Text Chunk No. 100", textSplit[150])
  return textSplit

**Step 05: Create Embeddings for each of the Text Chunks and Save these Embeddings into a Vector Database or Vector Store**

In [9]:
def create_embeddings_store(textSplit):
  model_name = "sentence-transformers/all-mpnet-base-v2"
  model_kwargs = {"device":"cuda"}
  embeddings = HuggingFaceEmbeddings(model_name = model_name, model_kwargs = model_kwargs)
  vectorStore = Chroma.from_documents(documents = textSplit, embedding = embeddings)
  return vectorStore

**Step 06: Retrieval and Generation**

In [10]:
def create_qa_chain(vectorStore):
  return RetrievalQA.from_chain_type(
      llm = ChatGroq(model = "llama-3.1-70b-versatile", temperature  = 0),
      chain_type = "stuff",
      retriever = vectorStore.as_retriever(search_type = "similarity", search_kwargs = {"k": 6})
  )

In [11]:
def processData(docx_path):
  text = extractText(docx_path)
  textSplit = textSplitter(text)
  vectorStore = create_embeddings_store(textSplit)
  qa_chain = create_qa_chain(vectorStore)
  return qa_chain

In [12]:
docx_path = '/content/Documents/'

In [13]:
qaChain = processData(docx_path)

Length of Text Chunks 178
Text Chunk No. 10 page_content='selection, propelling DETRs into the realm of real-time applications. Nevertheless, the inherent
complexity in deploying DETRs impedes its ability to attain the optimal balance between accuracy
and speed. Another line is to explore end-to-end detection for CNN-based detectors, which typically
leverages one-to-one assignment strategies to suppress the redundant predictions [ 5,49,60,73,16].
However, they usually introduce additional inference overhead or achieve suboptimal performance.
Furthermore, the model architecture design remains a fundamental challenge for YOLOs, which
exhibits an important impact on the accuracy and speed [ 45,16,65,7]. To achieve more efficient
and effective model architectures, researchers have explored different design strategies. Various
primary computational units are presented for the backbone to enhance the feature extraction ability,
including DarkNet [ 43,44,45], CSPNet [ 2], EfficientRep [ 27] a

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
qaChain.invoke({"query":"What advancements are made in YOLOv10"})

{'query': 'What advancements are made in YOLOv10',
 'result': 'According to the provided context, the advancements made in YOLOv10 include:\n\n1. **Improved accuracy**: YOLOv10 achieves state-of-the-art performance and end-to-end latency across various model scales.\n2. **Reduced parameters**: YOLOv10 has 28-57% fewer parameters compared to its baseline models (YOLOv8) and other YOLO models (e.g., YOLOv6-3.0, YOLOv9-C).\n3. **Reduced computations**: YOLOv10 has 23-38% less computations compared to its baseline models (YOLOv8) and other YOLO models (e.g., YOLOv6-3.0, YOLOv9-C).\n4. **Lower latency**: YOLOv10 has 37-70% lower latency compared to its baseline models (YOLOv8) and other YOLO models (e.g., YOLOv6-3.0, YOLOv9-C).\n5. **Faster inference speed**: YOLOv10-S/X achieves 1.8x/1.3x faster inference speed than RT-DETR-R18/R101, respectively, under similar performance.\n6. **Improved trade-offs between accuracy and computational cost**: YOLOv10 exhibits superior trade-offs between acc

In [15]:
qaChain.invoke({"query":"YOLOv7 is used for"})

{'query': 'YOLOv7 is used for',
 'result': 'According to the provided context, YOLOv7 (and its variants, such as YOLOv7-X, YOLOv7-W6, YOLOv7-E6, etc.) is used for object detection tasks.'}

In [21]:
qaChain.invoke({"query":"YOLOv7 outperforms which models"})

{'query': 'YOLOv7 outperforms which models',
 'result': 'According to the provided context, YOLOv7-X outperforms YOLOv5-X (r6.1) in terms of AP by 2.2%. Additionally, YOLOv7-X has a faster inference speed (31 fps faster) and reduces the number of parameters by 22% and computation by 8% compared to YOLOv5-X (r6.1).'}

In [23]:
qaChain.invoke({"query":"YOLOv7 performance comparison"})

{'query': 'YOLOv7 performance comparison',
 'result': 'Based on the provided context, here are some performance comparisons for YOLOv7:\n\n1. Compared to YOLOv5-X (r6.1):\n   - YOLOv7-X reduces 22% of parameters and 8% of computation.\n   - YOLOv7-X improves AP by 2.2%.\n   - YOLOv7-X has an inference speed 31 fps faster.\n\n2. Compared to YOLOv4:\n   - YOLOv7 has 75% fewer parameters.\n   - YOLOv7 has 36% less computation.\n   - YOLOv7 has 1.5% higher AP.\n\n3. Compared to YOLOR-CSP:\n   - YOLOv7 has 43% fewer parameters.\n   - YOLOv7 has 15% less computation.\n   - YOLOv7 has 0.4% higher AP.\n\n4. Compared to YOLOv4-tiny-31 (tiny model):\n   - YOLOv7-tiny reduces the number of parameters by 39%.\n   - YOLOv7-tiny reduces the amount of computation by 49%.\n   - YOLOv7-tiny maintains the same AP.\n\nNote that these comparisons are based on the provided context and may not be comprehensive or up-to-date.'}

In [27]:
qaChain.invoke({"query":"YOLOv10 is trained"})

{'query': 'YOLOv10 is trained',
 'result': 'YOLOv10 is trained on the COCO dataset.'}

In [28]:
qaChain.invoke({"query":"What advancements are made in YOLOv7"})

{'query': 'What advancements are made in YOLOv7',
 'result': 'Based on the provided context, the advancements made in YOLOv7 include:\n\n1. Improved accuracy: YOLOv7 achieves state-of-the-art performance, with AP improvements ranging from 0.3% to 1.5% compared to its baseline models.\n2. Reduced parameters: YOLOv7 reduces the number of parameters by 22% to 57% compared to its baseline models.\n3. Reduced computations: YOLOv7 reduces the number of computations by 8% to 38% compared to its baseline models.\n4. Improved speed: YOLOv7 achieves faster inference speeds, with a 31 fps improvement compared to YOLOv5-X (r6.1).\n5. Improved trade-offs between accuracy and computational cost: YOLOv7 exhibits superior trade-offs between accuracy and computational cost, making it suitable for various applications.\n6. Improved performance on real-time object detection: YOLOv7 surpasses all known object detectors in both speed and accuracy in the range from 5 FPS to 160 FPS and has the highest accur

In [29]:
qaChain.invoke({"query":"YOLOv10 uses NMS"})

{'query': 'YOLOv10 uses NMS',
 'result': 'That\'s incorrect. According to the text, YOLOv10 uses "NMS-free training with consistent dual assignments".'}